In [ ]:
from pathlib import Path
import subprocess

REPO_URL = "https://github.com/seoultechpse/fenicsx-colab.git"
ROOT = Path("/content")
REPO_DIR = ROOT / "fenicsx-colab"

subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)

USE_COMPLEX = False  # <--- Set True ONLY if you need complex PETSc
USE_CLEAN = False    # <--- Set True to remove existing environment

opts_str = " ".join(
  [o for c, o in [(USE_COMPLEX, "--complex"), (USE_CLEAN, "--clean")] if c]
)

get_ipython().run_line_magic("run", f"{REPO_DIR / 'setup_fenicsx.py'} {opts_str}")

🔧 FEniCSx Setup Configuration
PETSc type      : real
Clean install   : False

⚠️  Google Drive not mounted — using local cache (/content)

🔧 Installing FEniCSx environment...

🔍 Verifying PETSc type...
✅ Installed: Real PETSc (float64)

✨ Loading FEniCSx Jupyter magic... %%fenicsx registered

✅ FEniCSx setup complete!

Next steps:
  1. Run %%fenicsx --info to verify installation
  2. Use %%fenicsx in cells to run FEniCSx code
  3. Use -np N for parallel execution (e.g., %%fenicsx -np 4)

📌 Note: Real PETSc is installed
   - Recommended for most FEM problems
   - For complex problems, reinstall with --complex


---

In [ ]:
%%fenicsx

"""
Practical DOLFINx Integration Examples
======================================

This file shows how to use UFL elements in actual DOLFINx simulations.
These examples bridge the gap between UFL element definitions and solving PDEs.

Note: These examples require DOLFINx to be installed.
If DOLFINx is not available, they will show the setup without execution.

Based on FEniCS Workshop materials.
"""

# Check if DOLFINx is available
try:
    import dolfinx
    import dolfinx.fem
    import dolfinx.mesh
    from mpi4py import MPI
    DOLFINX_AVAILABLE = True
    print("DOLFINx is available - examples will run")
except ImportError:
    DOLFINX_AVAILABLE = False
    print("DOLFINx not available - showing setup only")

import basix.ufl
import ufl
import numpy as np

print("=" * 70)
print("PRACTICAL DOLFINX INTEGRATION EXAMPLES")
print("=" * 70)

# =============================================================================
# Example 1: Simple Poisson Problem
# =============================================================================
print("\n" + "=" * 70)
print("Example 1: Poisson Equation with Different Element Degrees")
print("=" * 70)

if DOLFINX_AVAILABLE:
    # Create a simple mesh
    mesh = dolfinx.mesh.create_unit_square(
        MPI.COMM_WORLD, 10, 10, dolfinx.mesh.CellType.triangle
    )
    print(f"Mesh created: {mesh.topology.index_map(2).size_local} cells")

    # Test different polynomial degrees
    for degree in [1, 2, 3, 4]:
        print(f"\n  Testing P{degree} Lagrange element:")

        # Create element
        element = basix.ufl.element("Lagrange", "triangle", degree)

        # Create function space
        V = dolfinx.fem.functionspace(mesh, element)

        print(f"    DOFs: {V.dofmap.index_map.size_global}")
        print(f"    DOFs per cell: {V.dofmap.dof_layout.num_dofs}")
else:
    print("\nSetup (DOLFINx not available):")
    print("  1. Create mesh: mesh = dolfinx.mesh.create_unit_square(...)")
    print("  2. Define element: element = basix.ufl.element('Lagrange', 'triangle', degree)")
    print("  3. Create function space: V = dolfinx.fem.functionspace(mesh, element)")

# =============================================================================
# Example 2: Vector-Valued Problem (Linear Elasticity)
# =============================================================================
print("\n" + "=" * 70)
print("Example 2: Linear Elasticity (Vector-Valued)")
print("=" * 70)

if DOLFINX_AVAILABLE:
    mesh = dolfinx.mesh.create_unit_square(
        MPI.COMM_WORLD, 8, 8, dolfinx.mesh.CellType.triangle
    )

    # Create vector element for displacement
    element = basix.ufl.element("Lagrange", "triangle", 2, shape=(2,))
    V = dolfinx.fem.functionspace(mesh, element)

    print(f"Vector function space created:")
    print(f"  Element: P2 Lagrange vector (2D)")
    print(f"  Total DOFs: {V.dofmap.index_map.size_global}")
    print(f"  DOFs per cell: {V.dofmap.dof_layout.num_dofs}")
    print(f"  Value shape: {V.element.value_shape}")

    # Create trial and test functions
    u = ufl.TrialFunction(V)
    v = ufl.TestFunction(V)
    print(f"\n  Trial function u created (displacement)")
    print(f"  Test function v created")
else:
    print("\nSetup (DOLFINx not available):")
    print("  1. element = basix.ufl.element('Lagrange', 'triangle', 2, shape=(2,))")
    print("  2. V = dolfinx.fem.functionspace(mesh, element)")
    print("  3. u = ufl.TrialFunction(V)  # Displacement")

# =============================================================================
# Example 3: Stokes Problem with Taylor-Hood Elements
# =============================================================================
print("\n" + "=" * 70)
print("Example 3: Stokes Flow with Taylor-Hood Elements")
print("=" * 70)

if DOLFINX_AVAILABLE:
    mesh = dolfinx.mesh.create_unit_square(
        MPI.COMM_WORLD, 8, 8, dolfinx.mesh.CellType.triangle
    )

    # Create Taylor-Hood mixed element
    P2 = basix.ufl.element("Lagrange", "triangle", 2, shape=(2,))
    P1 = basix.ufl.element("Lagrange", "triangle", 1)
    TH = basix.ufl.mixed_element([P2, P1])

    # Create mixed function space
    W = dolfinx.fem.functionspace(mesh, TH)

    print(f"Mixed function space (Taylor-Hood) created:")
    print(f"  Velocity: P2 vector")
    print(f"  Pressure: P1 scalar")
    print(f"  Total DOFs: {W.dofmap.index_map.size_global}")

    # Split the mixed space
    num_subs = W.element.num_sub_elements
    print(f"  Number of subspaces: {num_subs}")

    # Create trial and test functions
    (u, p) = ufl.TrialFunctions(W)
    (v, q) = ufl.TestFunctions(W)
    print(f"\n  Trial functions: u (velocity), p (pressure)")
    print(f"  Test functions: v, q")
else:
    print("\nSetup (DOLFINx not available):")
    print("  1. P2 = basix.ufl.element('Lagrange', 'triangle', 2, shape=(2,))")
    print("  2. P1 = basix.ufl.element('Lagrange', 'triangle', 1)")
    print("  3. TH = basix.ufl.mixed_element([P2, P1])")
    print("  4. W = dolfinx.fem.functionspace(mesh, TH)")
    print("  5. (u, p) = ufl.TrialFunctions(W)")

# =============================================================================
# Example 4: Stokes with MINI Elements
# =============================================================================
print("\n" + "=" * 70)
print("Example 4: Stokes Flow with MINI Elements")
print("=" * 70)

if DOLFINX_AVAILABLE:
    mesh = dolfinx.mesh.create_unit_square(
        MPI.COMM_WORLD, 8, 8, dolfinx.mesh.CellType.triangle
    )

    # Create MINI element
    P1_elem = basix.ufl.element("Lagrange", "triangle", 1)
    B_elem = basix.ufl.element("Bubble", "triangle", 3)
    enriched = basix.ufl.enriched_element([P1_elem, B_elem])
    V_elem = basix.ufl.blocked_element(enriched, shape=(2,))
    Q_elem = basix.ufl.element("Lagrange", "triangle", 1)
    MINI = basix.ufl.mixed_element([V_elem, Q_elem])

    # Create mixed function space
    W = dolfinx.fem.functionspace(mesh, MINI)

    print(f"Mixed function space (MINI) created:")
    print(f"  Velocity: P1 + Bubble, blocked to vector")
    print(f"  Pressure: P1 scalar")
    print(f"  Total DOFs: {W.dofmap.index_map.size_global}")
    print(f"  Fewer DOFs than Taylor-Hood for same mesh")
else:
    print("\nSetup (DOLFINx not available):")
    print("  1. Create enriched element: P1 + Bubble")
    print("  2. Block to vector: basix.ufl.blocked_element(..., shape=(2,))")
    print("  3. Create mixed element with P1 pressure")
    print("  4. W = dolfinx.fem.functionspace(mesh, MINI)")

# =============================================================================
# Example 5: DG Method for Advection
# =============================================================================
print("\n" + "=" * 70)
print("Example 5: Discontinuous Galerkin for Advection")
print("=" * 70)

if DOLFINX_AVAILABLE:
    mesh = dolfinx.mesh.create_unit_square(
        MPI.COMM_WORLD, 10, 10, dolfinx.mesh.CellType.triangle
    )

    # Test different DG orders
    for degree in [0, 1, 2]:
        element = basix.ufl.element("DG", "triangle", degree)
        V = dolfinx.fem.functionspace(mesh, element)

        print(f"\n  DG{degree} space:")
        print(f"    Total DOFs: {V.dofmap.index_map.size_global}")
        print(f"    DOFs per cell: {V.dofmap.dof_layout.num_dofs}")
else:
    print("\nSetup (DOLFINx not available):")
    print("  1. element = basix.ufl.element('DG', 'triangle', degree)")
    print("  2. V = dolfinx.fem.functionspace(mesh, element)")
    print("  Note: All DOFs are interior to cells (no coupling)")

# =============================================================================
# Example 6: Curved Domain with Higher-Order Geometry
# =============================================================================
print("\n" + "=" * 70)
print("Example 6: Higher-Order Geometry (Curved Mesh)")
print("=" * 70)

if DOLFINX_AVAILABLE:
    # Create mesh with quadratic geometry
    mesh = dolfinx.mesh.create_unit_square(
        MPI.COMM_WORLD, 5, 5, dolfinx.mesh.CellType.triangle
    )

    # The mesh itself can be converted to higher order
    print(f"Mesh created with linear geometry (P1)")
    print(f"  Can represent straight edges only")

    # For curved boundaries, you would typically:
    # 1. Create mesh with higher-order coordinate element
    # 2. Or use external mesh generators (e.g., Gmsh) with curved edges

    print(f"\n  To create truly curved mesh:")
    print(f"    - Use Gmsh with higher-order elements")
    print(f"    - Or manually set higher-order coordinates")
else:
    print("\nSetup (DOLFINx not available):")
    print("  For curved geometries:")
    print("  1. Generate mesh in Gmsh with curved elements")
    print("  2. Import with dolfinx.io.gmshio.read_from_msh()")
    print("  3. The coordinate element determines curvature")

# =============================================================================
# Example 7: Sub-parametric vs Iso-parametric
# =============================================================================
print("\n" + "=" * 70)
print("Example 7: Comparing Parametrizations")
print("=" * 70)

if DOLFINX_AVAILABLE:
    # Linear mesh
    mesh = dolfinx.mesh.create_unit_square(
        MPI.COMM_WORLD, 6, 6, dolfinx.mesh.CellType.triangle
    )

    print(f"Mesh: Linear triangles (P1 geometry)")
    print(f"  {mesh.topology.index_map(2).size_local} cells")

    # Iso-parametric: P1 solution on P1 mesh
    iso_elem = basix.ufl.element("Lagrange", "triangle", 1)
    V_iso = dolfinx.fem.functionspace(mesh, iso_elem)
    print(f"\n  Iso-parametric (P1 on P1 mesh):")
    print(f"    DOFs: {V_iso.dofmap.index_map.size_global}")

    # Sub-parametric: P3 solution on P1 mesh
    sub_elem = basix.ufl.element("Lagrange", "triangle", 3)
    V_sub = dolfinx.fem.functionspace(mesh, sub_elem)
    print(f"\n  Sub-parametric (P3 on P1 mesh):")
    print(f"    DOFs: {V_sub.dofmap.index_map.size_global}")
    print(f"    More DOFs → Better approximation")
else:
    print("\nConcept:")
    print("  Iso-parametric: Same degree for mesh and solution")
    print("  Sub-parametric: Higher degree for solution (common)")
    print("  Super-parametric: Lower degree for solution (rare)")

# =============================================================================
# Example 8: Complete Variational Form Setup
# =============================================================================
print("\n" + "=" * 70)
print("Example 8: Complete Variational Form (Poisson)")
print("=" * 70)

if DOLFINX_AVAILABLE:
    mesh = dolfinx.mesh.create_unit_square(
        MPI.COMM_WORLD, 10, 10, dolfinx.mesh.CellType.triangle
    )

    # Create function space
    element = basix.ufl.element("Lagrange", "triangle", 2)
    V = dolfinx.fem.functionspace(mesh, element)

    # Define trial and test functions
    u = ufl.TrialFunction(V)
    v = ufl.TestFunction(V)

    # Define source term
    f = dolfinx.fem.Constant(mesh, 1.0)

    # Define variational problem: -∆u = f
    a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
    L = f * v * ufl.dx

    print(f"Variational formulation created:")
    print(f"  Problem: -∆u = f")
    print(f"  a(u,v) = ∫ ∇u·∇v dx")
    print(f"  L(v) = ∫ f v dx")
    print(f"  Function space: P2 Lagrange")
    print(f"  DOFs: {V.dofmap.index_map.size_global}")

    print(f"\n  Next steps would be:")
    print(f"    1. Apply boundary conditions")
    print(f"    2. Assemble system matrix and vector")
    print(f"    3. Solve linear system")
else:
    print("\nSetup (DOLFINx not available):")
    print("  V = dolfinx.fem.functionspace(mesh, element)")
    print("  u = ufl.TrialFunction(V)")
    print("  v = ufl.TestFunction(V)")
    print("  a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx")
    print("  L = f * v * ufl.dx")

# =============================================================================
# Example 9: Element Information Query
# =============================================================================
print("\n" + "=" * 70)
print("Example 9: Querying Element Properties")
print("=" * 70)

if DOLFINX_AVAILABLE:
    mesh = dolfinx.mesh.create_unit_square(
        MPI.COMM_WORLD, 4, 4, dolfinx.mesh.CellType.triangle
    )

    # Create various elements and query properties
    elements = {
        "P1 Scalar": basix.ufl.element("Lagrange", "triangle", 1),
        "P2 Vector": basix.ufl.element("Lagrange", "triangle", 2, shape=(2,)),
        "DG1": basix.ufl.element("DG", "triangle", 1),
        "N1curl": basix.ufl.element("N1curl", "triangle", 1),
    }

    for name, elem in elements.items():
        V = dolfinx.fem.functionspace(mesh, elem)
        print(f"\n  {name}:")
        print(f"    Element: {elem}")
        print(f"    Value shape: {V.element.value_shape}")
        print(f"    DOFs per cell: {V.dofmap.dof_layout.num_dofs}")
        print(f"    Total DOFs: {V.dofmap.index_map.size_global}")
else:
    print("\nElement properties can be queried:")
    print("  - Value shape (scalar, vector, tensor)")
    print("  - DOFs per cell")
    print("  - Total DOFs in function space")
    print("  - Element family and degree")

# =============================================================================
# Example 10: Practical Workflow Summary
# =============================================================================
print("\n" + "=" * 70)
print("Example 10: Typical DOLFINx Workflow")
print("=" * 70)

workflow = """
STEP-BY-STEP WORKFLOW:

1. CREATE OR LOAD MESH
   mesh = dolfinx.mesh.create_unit_square(...)
   # or load from file

2. DEFINE FINITE ELEMENT
   element = basix.ufl.element("Lagrange", "triangle", degree)
   # or mixed_element for coupled problems

3. CREATE FUNCTION SPACE
   V = dolfinx.fem.functionspace(mesh, element)

4. DEFINE TRIAL AND TEST FUNCTIONS
   u = ufl.TrialFunction(V)
   v = ufl.TestFunction(V)

5. DEFINE VARIATIONAL FORM
   a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
   L = f * v * ufl.dx

6. APPLY BOUNDARY CONDITIONS
   bc = dolfinx.fem.dirichletbc(...)

7. ASSEMBLE AND SOLVE
   problem = dolfinx.fem.petsc.LinearProblem(a, L, bcs=[bc])
   uh = problem.solve()

8. POST-PROCESS
   # Visualize, save, compute quantities of interest
"""

print(workflow)

# =============================================================================
# Summary
# =============================================================================
print("\n" + "=" * 70)
print("SUMMARY")
print("=" * 70)
print("""
Practical Integration Points:
1. UFL elements → DOLFINx function spaces
2. Element degree affects accuracy and cost
3. Mixed elements for coupled problems
4. DG elements for specific applications
5. Trial/test functions from function spaces
6. Variational forms use UFL operators
7. Complete workflow: mesh → element → space → form → solve

Key Functions:
- basix.ufl.element(): Define element
- dolfinx.fem.functionspace(): Create space
- ufl.TrialFunction(): Unknown function
- ufl.TestFunction(): Test function
- ufl.dx: Integration measure
- ufl.grad, ufl.div, ufl.inner: Operators
""")

if DOLFINX_AVAILABLE:
    print("\n✓ All examples executed with DOLFINx")
else:
    print("\n⚠ DOLFINx not available - showed setup only")
    print("  Install DOLFINx to run these examples")

print("\n" + "=" * 70)
print("Practical examples completed!")
print("=" * 70)

DOLFINx is available - examples will run
PRACTICAL DOLFINX INTEGRATION EXAMPLES

Example 1: Poisson Equation with Different Element Degrees
Mesh created: 200 cells

  Testing P1 Lagrange element:
    DOFs: 121
    DOFs per cell: 3

  Testing P2 Lagrange element:
    DOFs: 441
    DOFs per cell: 6

  Testing P3 Lagrange element:
    DOFs: 961
    DOFs per cell: 10

  Testing P4 Lagrange element:
    DOFs: 1681
    DOFs per cell: 15

Example 2: Linear Elasticity (Vector-Valued)
Vector function space created:
  Element: P2 Lagrange vector (2D)
  Total DOFs: 289
  DOFs per cell: 6
  Value shape: [2]

  Trial function u created (displacement)
  Test function v created

Example 3: Stokes Flow with Taylor-Hood Elements
Mixed function space (Taylor-Hood) created:
  Velocity: P2 vector
  Pressure: P1 scalar
  Total DOFs: 659
  Number of subspaces: 2

  Trial functions: u (velocity), p (pressure)
  Test functions: v, q

Example 4: Stokes Flow with MINI Elements
Mixed function space (MINI) create